# PyTorch🔥Embeddings Example

Subject: Building a model to create embeddings for the dataset's words representing their near closeness to each other. Training with two subsequent words as train and the subsequent third word as target (N-GRAMS).

Data: Static string (just as an example)

Procedure:
- Tokenizing simply via '...'.split()
- Creating contexts and targets from three words each: (0, 1) with (2) as target
- Creating a vocabulary as simple set of distinct words
- Tensorizing contexts and targets
- Word2vec-like CBOW model with torch.nn.module, torch.nn.Embedding, torch.nn.Linear,  and torch.nn.functional.log_softmax
- Training with torch.nn.NLLLoss, and torch.optim.SGD
- Evaluation by finding some nearest words and playing with word vectors
- Disappointing results (to be expected with that simple input data)

Sources used:
- https://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html

In [11]:
import heapq

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchtext

torch.manual_seed(1)

In [2]:
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10
LEARNING_RATE = 0.001
N_EPOCHS = 10

In [3]:
# We will use Shakespeare Sonnet 2
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()

In [4]:
# we should tokenize the input, but we will ignore that for now
# build a list of tuples.
ngrams: list[tuple[list[str], str]] = []
for i in range(CONTEXT_SIZE, len(test_sentence)):  # from 2 to 114, i.e. skipping first 0 and 1
    context_words = test_sentence[i-CONTEXT_SIZE:i]
    target_word = test_sentence[i]
    ngrams.append((context_words, target_word))

# Print the first 3, just so you can see what they look like.
print(ngrams[:3])

[(['When', 'forty'], 'winters'), (['forty', 'winters'], 'shall'), (['winters', 'shall'], 'besiege')]


## Build Vocabulary

In [5]:
vocab = set(test_sentence)
word_to_ix = {word: i for i, word in enumerate(vocab)}
print(word_to_ix)

# for manual evaluation later 
ix_to_word = [(i, word) for word, i in word_to_ix.items()]

{'child': 0, 'answer': 1, 'beauty': 2, 'To': 3, 'If': 4, 'old,': 5, 'now,': 6, 'And': 7, 'see': 8, 'it': 9, 'count,': 10, 'and': 11, 'more': 12, 'blood': 13, 'art': 14, 'Will': 15, 'worth': 16, 'be': 17, "deserv'd": 18, 'Where': 19, 'held:': 20, 'his': 21, 'my': 22, 'say,': 23, 'were': 24, 'a': 25, 'small': 26, 'dig': 27, 'warm': 28, 'where': 29, 'This': 30, "youth's": 31, 'forty': 32, 'being': 33, "excuse,'": 34, "totter'd": 35, "'This": 36, 'Shall': 37, 'own': 38, "beauty's": 39, 'of': 40, 'sum': 41, 'sunken': 42, 'lusty': 43, 'couldst': 44, 'Proving': 45, 'on': 46, 'much': 47, "feel'st": 48, 'Then': 49, 'lies,': 50, 'make': 51, 'the': 52, 'asked,': 53, 'so': 54, 'winters': 55, 'in': 56, 'livery': 57, 'all': 58, 'shame,': 59, 'shall': 60, 'use,': 61, 'thy': 62, 'thriftless': 63, 'eyes,': 64, 'Were': 65, 'field,': 66, 'made': 67, 'within': 68, 'old': 69, 'by': 70, 'treasure': 71, 'How': 72, 'Thy': 73, 'succession': 74, 'praise.': 75, 'fair': 76, 'When': 77, 'all-eating': 78, 'besiege'

## Model

In [6]:
class NGramLanguageModeler(nn.Module):

    def __init__(self, 
                 vocab_size: int, 
                 embedding_dim: int, 
                 context_size: int
                ):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(num_embeddings=vocab_size, 
                                       embedding_dim=embedding_dim
                                      )
        self.linear1 = nn.Linear(in_features=context_size * embedding_dim, 
                                 out_features=128)
        self.linear2 = nn.Linear(in_features=128, 
                                 out_features=vocab_size)

    def forward(self, inputs: torch.Tensor): # [2]
        embeds = self.embeddings(inputs).view((1, -1))  # [1, 20]
        out = F.relu(self.linear1(embeds))  # [1, 128]
        out = self.linear2(out)  # [1, 97]
        log_probs = F.log_softmax(out, dim=1)  # [1, 97]
        return log_probs

In [7]:
losses = []
loss_function = nn.NLLLoss()
model = NGramLanguageModeler(vocab_size=len(vocab), 
                             embedding_dim=EMBEDDING_DIM, 
                             context_size=CONTEXT_SIZE)
optimizer = optim.SGD(params=model.parameters(), 
                      lr=LEARNING_RATE)

for epoch in range(N_EPOCHS):
    total_loss = 0
    
    for context, target in ngrams:  # e.g. ['When', 'forty'] / 'winters'

        # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
        # into integer indices and wrap them in tensors)
        context_idxs = torch.tensor([word_to_ix[w] for w in context],  # [2]  e.g.[28, 21]
                                    dtype=torch.long)

        target_idx = torch.tensor([word_to_ix[target]], # [1], e.g. [81]
                                  dtype=torch.long)

        # Step 2. Recall that torch *accumulates* gradients. Before passing in a
        # new instance, you need to zero out the gradients from the old
        # instance
        model.zero_grad()

        # Step 3. Run the forward pass, getting log probabilities over next
        # words
        log_probs = model(context_idxs)  # [1, 97], i.e. a probability for each target word 

        # Step 4. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a tensor)
        loss = loss_function(log_probs,  # [], e.g. tensor(4.9681, grad_fn=<NllLossBackward0>)
                             target_idx)

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    losses.append(total_loss)

In [8]:
print(losses)  # The loss decreased every iteration over the training data!

# To get the embedding of a particular word, e.g. "beauty"
print(model.embeddings.weight[word_to_ix["beauty"]])

[520.9909830093384, 518.5847978591919, 516.1935036182404, 513.8159439563751, 511.45058369636536, 509.09558868408203, 506.7505099773407, 504.4150788784027, 502.0893211364746, 499.7709891796112]
tensor([ 0.8659,  0.2448, -0.6612,  0.8062,  1.1003, -0.1757, -2.2480, -1.4465,
         0.0606, -0.6189], grad_fn=<SelectBackward0>)


# Display the "nearest" and "most distant" words

Calculate the distance from each word to each other word

In [9]:
embeddings = list(model.embeddings.parameters())[0].detach().cpu()  # [{vocab_size}, {EMBEDDIG_DIM}] = [97, 10]

similarities: list[list[float]] = []  # will have length 97

# for all 97 words...
for word_index in range(len(embeddings)):

    # get the word's embedding (in all 10 embedding dimensions)
    x1 = embeddings[word_index].view(1,-1)  # [1, 10]
    
    similarities_to_current_word: list[float] = []  # will have length 97

    # for all 97 words...
    for other_word_index in range(len(embeddings)):

        # get the other word's embedding (in all 10 embedding dimensions)
        x2 = embeddings[other_word_index].view(1,-1)  # [1, 10]
        
        # compute similarity between the two words across all 10 embedding dimensions
        similarity = torch.cosine_similarity(x1=x1,   # similarity: float, e.g. 0.18973809480667114
                                             x2=x2).item()
        similarities_to_current_word.append(similarity)
    similarities.append(similarities_to_current_word)

Get the nearest and most distant word-to-word combinations

In [12]:
N_LARGEST = 10

flattened = [s for sim in similarities for s in sim]

# get indices of n largest elements
# the words themselves are always the most similar, thus we remove the first {voc_size}
n_largest = set(heapq.nlargest(len(vocab) + N_LARGEST, flattened))
n_largest_indices_ = []
for value in n_largest:
    # we can't use list.index() fn as multiple elements of flattened might have the same value
    indices = [i for i in range(len(flattened)) if flattened[i] == value]
    n_largest_indices_.extend(indices)
    
n_largest_indices = n_largest_indices_[len(vocab):]

print(n_largest_indices)


[5622, 9078, 8629, 9109, 2275, 4291, 459, 6891, 4057, 7801]


In [13]:
for index in n_largest_indices:
    # unravel to get the index in the 2d list
    row = index // 97
    col = index % 97
    assert flattened[index] == similarities[row][col]
    print(ix_to_word[row])
    print(ix_to_word[col])

(57, 'livery')
(93, 'gazed')
(93, 'gazed')
(57, 'livery')
(88, 'thine!')
(93, 'gazed')
(93, 'gazed')
(88, 'thine!')
(23, 'say,')
(44, 'couldst')
(44, 'couldst')
(23, 'say,')
(4, 'If')
(71, 'treasure')
(71, 'treasure')
(4, 'If')
(41, 'sum')
(80, 'when')
(80, 'when')
(41, 'sum')


In [14]:

print(len(similarities))

97


In [15]:
#import seaborn as sns
#import matplotlib.pyplot as plt
##fig = plt.figure()
#sns.heatmap(final, annot=True)
#display(fig)